In [1]:
%cd ..
    


/Users/eliaweiss/ai/segmentation/MobileSAM


In [2]:
from notebooks.Utils import non_max_suppression
from notebooks.Utils import create_random_color
from flaskUtil.MultiDetectors import MultiDetectors
import cv2
import numpy as np
font = cv2.FONT_HERSHEY_SIMPLEX 

from PIL import Image, ExifTags


04/23/2024 17:14:13 - INFO - timm.models._builder -   Loading pretrained weights from Hugging Face hub (timm/resnet50.a1_in1k)
04/23/2024 17:14:13 - INFO - timm.models._hub -   [timm/resnet50.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
Some weights of the model checkpoint at TahaDouaji/detr-doc-table-detection were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model

In [3]:
import os
folder = "/Users/eliaweiss/Documents/doc2txt/lineCv/1"
# folder = "/Users/eliaweiss/Documents/doc2txt/sihach/Invoices/3/img3"
file_list = os.listdir(folder)
file_list = [file for file in file_list if 
             ".jpg" in file 
             or ".png" in file 
             or ".jpeg" in file]
file_list = sorted(file_list, key=lambda x: int(
    os.path.splitext(os.path.basename(x))[0]))

In [4]:
fileIter = iter(file_list)
cc = 0

In [5]:



while True:
    file_path = next(fileIter)
    if cc < 0:
        cc+=1    
        continue
    cc+=1    
    
    print(cc)
    
    file_path = os.path.join(folder, file_path)
    print(file_path)

    image = Image.open(file_path)
    scores, boxes = MultiDetectors.multiDetector(image)
 
    tmpImg1 = np.array(image)    
    tmpImg1 = cv2.cvtColor(tmpImg1, cv2.COLOR_BGR2RGB)
    drawWidth = int(2 * tmpImg1.shape[0]*0.001)
    
    for score, box in zip(scores, boxes):
        box = [int(i) for i in box]
        score = f"{int(score*100)}"
        print("prob: ",score)
        l,b,r,t = box
        color = create_random_color()
        cv2.rectangle(tmpImg1,(l,t),(r,b),color,drawWidth)
   
        # Using cv2.putText() method 
        fontScale = drawWidth/2
        tmpImg1 = cv2.putText(tmpImg1, score, (l,t), font,  
                        fontScale, color, drawWidth) 
    cv2.imshow("nms",tmpImg1)    
    cv2.waitKey()
    

1
/Users/eliaweiss/Documents/doc2txt/lineCv/1/0.jpg

0: 640x480 (no detections), 642.8ms
Speed: 1.9ms preprocess, 642.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


prob:  10
prob:  33
prob:  11
prob:  32
prob:  24
prob:  51
prob:  95
prob:  10
prob:  21
prob:  66


KeyboardInterrupt: 